In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import os
import pandas as pd
from torch.utils.data import Dataset
from PIL import Image


In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [3]:
class CrackDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.labels_df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = transform
        self.label_map = {'Non-cracked': 0, 'Cracked': 1}

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = self.labels_df.iloc[idx, 0]
        label = self.label_map[self.labels_df.iloc[idx, 1]]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

In [4]:
# Load datasets
train_dataset = CrackDataset('../artifact_folder/train/labels.csv', '../artifact_folder/train/images', transform)
val_dataset = CrackDataset('../artifact_folder/val/labels.csv', '../artifact_folder/val/images', transform)
test_dataset = CrackDataset('../artifact_folder/test/labels.csv', '../artifact_folder/test/images', transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)


In [5]:
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # 2 classes: Cracked / Non-cracked


/Users/nanphattongsirisukool/Documents/GitHub/Structural-Defects-Network-MLOps/env/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/nanphattongsirisukool/Documents/GitHub/Structural-Defects-Network-MLOps/env/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [18]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()

        train_acc = correct / len(train_loader.dataset)
        print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}, Accuracy: {train_acc:.4f}")


In [20]:
def evaluate(model, val_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            correct += (outputs.argmax(1) == labels).sum().item()
    val_accuracy = correct / len(val_loader.dataset)
    print(f'Validation Accuracy: {val_accuracy:.4f}')


In [25]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)
evaluate(model, val_loader)


Epoch 1, Loss: 10595.1421, Accuracy: 0.6364
Epoch 2, Loss: 8369.4019, Accuracy: 0.7607
Epoch 3, Loss: 7632.7484, Accuracy: 0.7905
Epoch 4, Loss: 6988.4895, Accuracy: 0.8122
Epoch 5, Loss: 6575.0189, Accuracy: 0.8294
Epoch 6, Loss: 6196.2417, Accuracy: 0.8418
Epoch 7, Loss: 5777.6937, Accuracy: 0.8535
Epoch 8, Loss: 5029.1505, Accuracy: 0.8744
Epoch 9, Loss: 4336.9726, Accuracy: 0.8923
Epoch 10, Loss: 3418.7692, Accuracy: 0.9177
Validation Accuracy: 0.8733


In [27]:
torch.save(model.state_dict(), '../model/resnet_crack_classifier.pth')

In [28]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix
)

# Make sure model is in eval mode
model.eval()

# Consistent transform (same used during training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Evaluation function
def evaluate_model_on_split(split_name):
    base_path = f"../artifact_folder/{split_name}"
    labels_df = pd.read_csv(os.path.join(base_path, "labels.csv"))
    images_dir = os.path.join(base_path, "images")

    y_true, y_pred, y_score = [], [], []

    for _, row in labels_df.iterrows():
        img_path = os.path.join(images_dir, row["filename"])
        image = Image.open(img_path).convert("RGB")
        input_tensor = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            pred = output.argmax(dim=1).item()
            prob = torch.softmax(output, dim=1)[0][1].item()

        label = 1 if row["label"].lower() == "cracked" else 0
        y_true.append(label)
        y_pred.append(pred)
        y_score.append(prob)

    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, zero_division=0),
        "Recall": recall_score(y_true, y_pred, zero_division=0),
        "F1-Score": f1_score(y_true, y_pred, zero_division=0),
        "AUC-ROC": roc_auc_score(y_true, y_score),
        "Confusion Matrix": confusion_matrix(y_true, y_pred).tolist()
    }

# Run on all sets
for split in ['train', 'val', 'test']:
    print(f"📊 Evaluation for {split.upper()}")
    metrics = evaluate_model_on_split(split)
    for k, v in metrics.items():
        print(f"{k}: {v}")
    print()


📊 Evaluation for TRAIN
Accuracy: 0.934052333804809
Precision: 0.9719338715878508
Recall: 0.8939179632248939
F1-Score: 0.9312948977712285
AUC-ROC: 0.9850882188643193
Confusion Matrix: [[8265, 219], [900, 7584]]

📊 Evaluation for VAL
Accuracy: 0.8732913348389397
Precision: 0.5607424071991001
Recall: 0.7776911076443058
F1-Score: 0.6516339869281046
AUC-ROC: 0.9078117100283506
Confusion Matrix: [[6350, 781], [285, 997]]

📊 Evaluation for TEST
Accuracy: 0.8680926916221033
Precision: 0.5438100621820238
Recall: 0.7604743083003953
F1-Score: 0.6341463414634146
AUC-ROC: 0.8975873296663811
Confusion Matrix: [[6343, 807], [303, 962]]



In [29]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)


/Users/nanphattongsirisukool/Documents/GitHub/Structural-Defects-Network-MLOps/env/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [35]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler=None, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = correct / len(train_loader.dataset)

        # Evaluate on validation set
        model.eval()
        val_loss = 0.0
        val_correct = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                val_correct += (outputs.argmax(1) == labels).sum().item()

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct / len(val_loader.dataset)

        # Step the scheduler if provided
        if scheduler:
            scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{epochs} | "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")


In [36]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=10)


Epoch 1/10 | Train Loss: 0.3190 | Train Acc: 0.9336 | Val Loss: 0.5962 | Val Acc: 0.7551
Epoch 2/10 | Train Loss: 0.3091 | Train Acc: 0.9381 | Val Loss: 0.4578 | Val Acc: 0.8475
Epoch 3/10 | Train Loss: 0.3013 | Train Acc: 0.9439 | Val Loss: 0.5146 | Val Acc: 0.8432
Epoch 4/10 | Train Loss: 0.2876 | Train Acc: 0.9529 | Val Loss: 0.4240 | Val Acc: 0.8695
Epoch 5/10 | Train Loss: 0.2769 | Train Acc: 0.9587 | Val Loss: 0.4575 | Val Acc: 0.8590
Epoch 6/10 | Train Loss: 0.2774 | Train Acc: 0.9570 | Val Loss: 0.3901 | Val Acc: 0.8906
Epoch 7/10 | Train Loss: 0.2701 | Train Acc: 0.9603 | Val Loss: 0.3726 | Val Acc: 0.8985
Epoch 8/10 | Train Loss: 0.2639 | Train Acc: 0.9645 | Val Loss: 0.4373 | Val Acc: 0.8669
Epoch 9/10 | Train Loss: 0.2604 | Train Acc: 0.9673 | Val Loss: 0.5067 | Val Acc: 0.8191
Epoch 10/10 | Train Loss: 0.2617 | Train Acc: 0.9658 | Val Loss: 0.5446 | Val Acc: 0.8041


In [37]:
torch.save(model.state_dict(), '../model/resnet_cos_crack_classifier.pth')

In [38]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix
)

# Make sure model is in eval mode
model.eval()

# Consistent transform (same used during training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Evaluation function
def evaluate_model_on_split(split_name):
    base_path = f"../artifact_folder/{split_name}"
    labels_df = pd.read_csv(os.path.join(base_path, "labels.csv"))
    images_dir = os.path.join(base_path, "images")

    y_true, y_pred, y_score = [], [], []

    for _, row in labels_df.iterrows():
        img_path = os.path.join(images_dir, row["filename"])
        image = Image.open(img_path).convert("RGB")
        input_tensor = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            pred = output.argmax(dim=1).item()
            prob = torch.softmax(output, dim=1)[0][1].item()

        label = 1 if row["label"].lower() == "cracked" else 0
        y_true.append(label)
        y_pred.append(pred)
        y_score.append(prob)

    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, zero_division=0),
        "Recall": recall_score(y_true, y_pred, zero_division=0),
        "F1-Score": f1_score(y_true, y_pred, zero_division=0),
        "AUC-ROC": roc_auc_score(y_true, y_score),
        "Confusion Matrix": confusion_matrix(y_true, y_pred).tolist()
    }

# Run on all sets
for split in ['train', 'val', 'test']:
    print(f"📊 Evaluation for {split.upper()}")
    metrics = evaluate_model_on_split(split)
    for k, v in metrics.items():
        print(f"{k}: {v}")
    print()


📊 Evaluation for TRAIN
Accuracy: 0.9738920320603489
Precision: 0.9624985620614287
Recall: 0.9862093352192363
F1-Score: 0.9742096990161262
AUC-ROC: 0.9971925132501126
Confusion Matrix: [[8158, 326], [117, 8367]]

📊 Evaluation for VAL
Accuracy: 0.8041126827528824
Precision: 0.4283476898981989
Recall: 0.8533541341653667
F1-Score: 0.570385818561001
AUC-ROC: 0.9042903575629774
Confusion Matrix: [[5671, 1460], [188, 1094]]

📊 Evaluation for TEST
Accuracy: 0.8046345811051694
Precision: 0.42518752467429927
Recall: 0.8513833992094861
F1-Score: 0.5671406003159558
AUC-ROC: 0.9050721689377816
Confusion Matrix: [[5694, 1456], [188, 1077]]



In [39]:
from sklearn.metrics import f1_score

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler=None, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = correct / len(train_loader.dataset)

        # Evaluate on validation set
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)

                preds = outputs.argmax(1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss /= len(val_loader.dataset)
        val_f1 = f1_score(all_labels, all_preds)

        if scheduler:
            scheduler.step(val_loss)  # Optionally use val_loss or convert to custom scheduler

        print(f"Epoch {epoch+1}/{epochs} | "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f} | Val F1: {val_f1:.4f}")


In [ ]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=10)


In [11]:
# Load the state_dict
model.load_state_dict(torch.load('../model/resnet_cos_crack_classifier.pth'))

# Set model to evaluation mode
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [12]:
class SoftF1Loss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, logits, targets):
        probs = torch.softmax(logits, dim=1)[:, 1]
        targets = targets.float()

        TP = (probs * targets).sum()
        FP = (probs * (1 - targets)).sum()
        FN = ((1 - probs) * targets).sum()

        soft_f1 = 2 * TP / (2 * TP + FP + FN + 1e-8)
        return 1 - soft_f1  # we want to minimize loss, so use 1 - F1


In [13]:
from sklearn.metrics import f1_score

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler=None, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = correct / len(train_loader.dataset)

        # Evaluate on validation set
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)

                preds = outputs.argmax(1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss /= len(val_loader.dataset)
        val_f1 = f1_score(all_labels, all_preds)

        if scheduler:
            scheduler.step(val_loss)  # Optionally use val_loss or convert to custom scheduler

        print(f"Epoch {epoch+1}/{epochs} | "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f} | Val F1: {val_f1:.4f}")


In [14]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

criterion = SoftF1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=10)

/Users/nanphattongsirisukool/Documents/GitHub/Structural-Defects-Network-MLOps/env/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/10 | Train Loss: 0.0759 | Train Acc: 0.9298 | Val Loss: 0.4339 | Val F1: 0.5826
Epoch 2/10 | Train Loss: 0.0978 | Train Acc: 0.9058 | Val Loss: 0.4222 | Val F1: 0.6001
Epoch 3/10 | Train Loss: 0.1043 | Train Acc: 0.9005 | Val Loss: 0.5128 | Val F1: 0.4996
Epoch 4/10 | Train Loss: 0.1108 | Train Acc: 0.8922 | Val Loss: 0.4658 | Val F1: 0.5529
Epoch 5/10 | Train Loss: 0.1154 | Train Acc: 0.8878 | Val Loss: 0.4495 | Val F1: 0.5659
Epoch 6/10 | Train Loss: 0.0799 | Train Acc: 0.9240 | Val Loss: 0.4063 | Val F1: 0.6137
Epoch 7/10 | Train Loss: 0.0554 | Train Acc: 0.9477 | Val Loss: 0.4320 | Val F1: 0.5847
Epoch 8/10 | Train Loss: 0.0463 | Train Acc: 0.9567 | Val Loss: 0.4270 | Val F1: 0.5937
Epoch 9/10 | Train Loss: 0.0364 | Train Acc: 0.9661 | Val Loss: 0.4643 | Val F1: 0.5515
Epoch 10/10 | Train Loss: 0.0245 | Train Acc: 0.9781 | Val Loss: 0.3754 | Val F1: 0.6479


In [15]:
torch.save(model.state_dict(), '../model/resnet_SoftF1Loss_crack_classifier.pth')

In [16]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix
)

# Make sure model is in eval mode
model.eval()

# Consistent transform (same used during training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Evaluation function
def evaluate_model_on_split(split_name):
    base_path = f"../artifact_folder/{split_name}"
    labels_df = pd.read_csv(os.path.join(base_path, "labels.csv"))
    images_dir = os.path.join(base_path, "images")

    y_true, y_pred, y_score = [], [], []

    for _, row in labels_df.iterrows():
        img_path = os.path.join(images_dir, row["filename"])
        image = Image.open(img_path).convert("RGB")
        input_tensor = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            pred = output.argmax(dim=1).item()
            prob = torch.softmax(output, dim=1)[0][1].item()

        label = 1 if row["label"].lower() == "cracked" else 0
        y_true.append(label)
        y_pred.append(pred)
        y_score.append(prob)

    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, zero_division=0),
        "Recall": recall_score(y_true, y_pred, zero_division=0),
        "F1-Score": f1_score(y_true, y_pred, zero_division=0),
        "AUC-ROC": roc_auc_score(y_true, y_score),
        "Confusion Matrix": confusion_matrix(y_true, y_pred).tolist()
    }

# Run on all sets
for split in ['train', 'val', 'test']:
    print(f"📊 Evaluation for {split.upper()}")
    metrics = evaluate_model_on_split(split)
    for k, v in metrics.items():
        print(f"{k}: {v}")
    print()


📊 Evaluation for TRAIN
Accuracy: 0.9892739273927392
Precision: 0.9948736289938007
Recall: 0.9836162187647336
F1-Score: 0.9892128971076339
AUC-ROC: 0.998387193487989
Confusion Matrix: [[8441, 43], [139, 8345]]

📊 Evaluation for VAL
Accuracy: 0.8722215618685368
Precision: 0.5584415584415584
Recall: 0.7714508580343213
F1-Score: 0.647887323943662
AUC-ROC: 0.896638482283086
Confusion Matrix: [[6349, 782], [293, 989]]

📊 Evaluation for TEST
Accuracy: 0.8762923351158646
Precision: 0.5645905420991926
Recall: 0.7739130434782608
F1-Score: 0.6528842947649216
AUC-ROC: 0.8967909560794937
Confusion Matrix: [[6395, 755], [286, 979]]

